## Data collection toolkit for monitoring "peer effects"

This cell sets up the notebook to import numpy, datascience, seaborn, pandas, matplotlib etc.

In [71]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Datascience, pandas modules.
import numpy as np
import pandas as pd
import seaborn as sns
from datascience import *
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

Defining the class `Person`

In [72]:
class Person:
    def __init__(self, index, startTime):
        self.index = index
        self.startTime = startTime
        self.breakStart = []
        self.breakEnd = []
        self.endTime = None
    
    def addBreak(self, time):
        self.breakStart.append(time)
        return "Break started for person " + str(self.index) + " at " + str(time)
    
    def endBreak(self, time):
        if (len(self.breakStart)-1)!=len(self.breakEnd):
            return "Please start a break to stop it."
        else:
            self.breakEnd.append(time)
            return "Break ended for person " + str(self.index) + " at " + str(time)
        
    def endSession(self, time):
        self.endTime = time
        return "Person " + str(self.index) + " left at " + str(time)
        
    def numBreaks(self):
        return len(self.breakEnd)

Defining the class `Table`

In [73]:
class Table:
    listTables = []
    
    def __init__(self, name, libraryName = '', people = 0):
        self.name = name
        self.startTime = dt.datetime.now()
        self.library = libraryName
        self.persons = []
        for i in range(1, people+1):
            self.addPerson()
        Table.listTables.append(self)
        print("Table with name " + str(name) + " and " + str(people) + " people" + " created!")
        print("Time: " + str(self.startTime))
        
    def addPerson(self):
        number = len(self.persons)+1
        self.persons.append(Person(number, dt.datetime.now()))
        print("Person number " + str(number) + " added.")
        return
    
    def addBreak(self, name):
        if name > len(self.persons):
            return "Person " + str(name) + " not in list of people."
        person = self.persons[name-1]
        time = dt.datetime.now()
        print(person.addBreak(time))
        
    def endBreak(self, name):
        if name > len(self.persons):
            return "Person " + str(name) + " not in list of people."
        time = dt.datetime.now()
        person = self.persons[name-1]
        print(person.endBreak(time))
        
    def removePerson(self, name):
        exists = False
        for person in self.persons:
            if (person.index == name) and (person.endTime == None):
                exists = True
                break
        if (not exists):
            print("This person doesn't exist and so, can't be removed")
            return
        else:
            time = dt.datetime.now()
            print(person.endSession(time))
            return
    
    def getNumBreaks(self, name):
        person = self.persons[name-1]
        print("Person " + str(name) + " has taken " + str(person.numBreaks()) + " breaks.")
        return

Creating the `convertToRow` function for appending to Dataframe

In [88]:
def convertToRow(person, table):
    row = [person.index, table.name, table.library, person.startTime, \
           person.endTime, person.numBreaks(), person.breakStart, person.breakEnd]
    neighbors = [person_p.index for person_p in table.persons if person_p.index!=person.index]
    row = row + [neighbors]
    return row

Creating a wrapper class `ListTables`

**Features:**
* Person departure
* Person arrival
* Person break begins
* Person checking phone

### Demo

In [74]:
# Creates a new table named 1 in Moffit Library
A = Table("A", "Moffit", 4)

# Adds person 5 to table 1
A.addPerson()

Person number 1 added.
Person number 2 added.
Person number 3 added.
Person number 4 added.
Table with name A and 4 people created!
Time: 2019-02-20 00:45:27.294338
Person number 5 added.


In [75]:
# Person 5 on Table A leaves for a break
A.addBreak(5)

Break started for person 5 at 2019-02-20 00:45:27.620293


In [76]:
B = Table("B", "Stacks", 3)
C = Table("C", "Moffit", 1)

# Person 2 and 3 on Table B leave together for a break
B.addBreak(2)
B.addBreak(3)

Person number 1 added.
Person number 2 added.
Person number 3 added.
Table with name B and 3 people created!
Time: 2019-02-20 00:45:28.062815
Person number 1 added.
Table with name C and 1 people created!
Time: 2019-02-20 00:45:28.063145
Break started for person 2 at 2019-02-20 00:45:28.063249
Break started for person 3 at 2019-02-20 00:45:28.063306


In [77]:
# Person 5 on Table A comes back
A.endBreak(5)

# Person 2 and 3 on Table B come back
B.endBreak(2)
B.endBreak(3)

Break ended for person 5 at 2019-02-20 00:45:28.514386
Break ended for person 2 at 2019-02-20 00:45:28.514486
Break ended for person 3 at 2019-02-20 00:45:28.514547


In [78]:
# A5 leaves for break and comes back again
A.addBreak(5)
A.endBreak(5)

Break started for person 5 at 2019-02-20 00:45:28.978167
Break ended for person 5 at 2019-02-20 00:45:28.978269


In [79]:
# Getting number of breaks taken by A5
A.getNumBreaks(5)

Person 5 has taken 2 breaks.


In [80]:
A.removePerson(5)

Person 5 left at 2019-02-20 00:45:29.706721


**Format:** Person, Table, Library, Arrival Time, Departure Time, Num Breaks, Break Start Times (list), Break End Times (list), Neighbours, Known

### Important (Read me)
1. Add the names of the tables that you want to add to the **database file** to the **list**
2. Run the cell below

In [90]:
# ADD the names of the required tables to this list,
# For example, required_tables = [A, B]

required_tables = [A, B, C]

In [94]:
# RUN THIS CELL AT THE END, then close this tab!

# This automatically adds data that we've input into the libData.csv dataframe
df = pd.read_csv('libData.csv')
for table in required_tables:
    for person in table.persons:
        df = df.append(pd.DataFrame([convertToRow(person, table)],columns=df.columns))
df.to_csv('libData.csv', index=False)